In [1]:
#importing the necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
import chardet
import glob
import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams["figure.figsize"] = [15, 10]
from bertopic import BERTopic
from umap import UMAP
import random
random.seed(42)

In [2]:
#avoid any stochastic behaviour in bertopic
umap_model = UMAP(n_neighbors=15, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=42)

In [4]:
# read all request files
pedidos_path = glob.glob("../data/20220219_Pedidos_csv_*")

files_list = []

for filename in pedidos_path:
    
    pedidos_file = pd.read_csv(filename,
                               sep=";",
                               quotechar="'",
                        header = None,
                        names=['IdPedido',
                              'ProtocoloPedido',
                              'Esfera',
                              'OrgaoDestinatario',
                              'Situacao',
                              'DataRegistro',
                              'ResumoSolicitacao',
                              'DetalhamentoSolicitacao',
                              'PrazoAtendimento',
                              'FoiProrrogado',
                              'FoiReencaminhado',
                              'FormaResposta',
                              'OrigemSolicitacao',
                              'IdSolicitante',
                              'AssuntoPedido',
                              'SubAssuntoPedido',
                              'Tag',
                              'DataResposta',
                              'Resposta',
                              'Decisao',
                              'EspecificacaoDecisao'],
                               keep_default_na=False,
                        encoding='UTF-16 LE')
    
    files_list.append(pedidos_file)

pedidos = pd.concat(files_list, axis=0, ignore_index=True)

In [5]:
#formatting date columns
pedidos['DataRegistro_formatted'] =  pd.to_datetime(pedidos.DataRegistro,errors='coerce', format="%d/%m/%Y")
pedidos['PrazoAtendimento_formatted'] =  pd.to_datetime(pedidos.PrazoAtendimento, errors='coerce',format="%d/%m/%Y")
pedidos['DataResposta_formatted'] =  pd.to_datetime(pedidos.DataResposta,errors='coerce', format="%d/%m/%Y")

In [11]:
pedidos['lgpd_flag'] = pedidos.Resposta.apply(lambda x: 'lgpd' in x or '13.709' in x or 'lpgd' in x)

In [12]:
pedidos['lgpd_flag'].sum()

1679

In [26]:
pedidos_lgpd = pedidos[pedidos['lgpd_flag']]

In [27]:
pedidos_list = pedidos_lgpd.DetalhamentoSolicitacao.to_list()

In [66]:
topic_model_v1 = BERTopic(verbose=True, language='Portuguese', nr_topics="auto", umap_model=umap_model)

In [67]:
requests_topics, requests_probabilities = topic_model_v1.fit_transform(pedidos_list)

Batches:   0%|          | 0/53 [00:00<?, ?it/s]

2022-02-21 11:42:31,000 - BERTopic - Transformed documents to Embeddings
2022-02-21 11:42:37,935 - BERTopic - Reduced dimensionality with UMAP
2022-02-21 11:42:38,358 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2022-02-21 11:42:42,283 - BERTopic - Reduced number of topics from 29 to 19


In [69]:
topic_model_v1.get_topic_freq()

,Topic,Count
0,0,641
1,-1,275
2,1,238
3,2,136
4,3,65
5,4,43
6,5,39
7,6,28
8,7,24
9,8,23


In [70]:
pedidos_lgpd['requests_topics'] = requests_topics
pedidos_lgpd['requests_probabilities'] = requests_probabilities

In [71]:
for t in set(requests_topics):
    pedidos_lgpd.loc[pedidos_lgpd.requests_topics == t,
                              'requests_topics_top_words'] = \
    str([a_tuple[0] for a_tuple in topic_model_v1.get_topic(t)])[1:-1]

In [72]:
pedidos_lgpd[['requests_topics','requests_topics_top_words']].drop_duplicates().sort_values('requests_topics')

,requests_topics,requests_topics_top_words
99889,-1,"'de', 'do', 'que', 'da', 'em', 'ao', 'no', 'processo', 'com', 'acesso'"
63362,0,"'de', 'da', 'por', 'do', 'dados', 'que', 'em', 'para', 'data', 'os'"
127167,1,"'processo', 'incra', 'acesso', 'sei', '54000', 'nº', 'ao', 'do', 'de', 'administrativo'"
128095,2,"'meu', 'inss', 'não', 'que', 'extrato', 'saber', 'eu', 'do', 'receber', 'benefício'"
190644,3,"'entrada', 'saída', 'registros', 'palácio', 'planalto', 'de', 'da', 'do', 'solicito', 'bolsonaro'"
101258,4,"'políticas', 'que', 'em', 'entrará', 'determinado', 'vigor', 'existem', 'positivo', 'possui', 'informação'"
188852,5,"'54000', 'processo', '2021', 'andamento', '2020', 'saber', 'do', 'gostaria', '08', 'comercial'"
190325,6,"'lgpd', 'tratamento', 'de', 'dados', 'ou', 'foi', 'adequação', 'pessoais', 'anpd', 'da'"
161965,7,"'dados', 'pessoais', 'meus', 'base', 'de', 'com', 'informação', 'dos', 'do', 'da'"
188572,8,"'imbel', 'pistola', 'gc', 'md2', 'armamento', 'gostaria', '45', 'saber', 'da', 'de'"


In [43]:
pedidos_respostas_list = pedidos_lgpd.Resposta.to_list()
topic_model_v2 = BERTopic(verbose=True, language='Portuguese', nr_topics="auto", umap_model=umap_model)

In [44]:
requests_topics, requests_probabilities = topic_model_v2.fit_transform(pedidos_respostas_list)

Batches:   0%|          | 0/53 [00:00<?, ?it/s]

2022-02-20 14:51:16,404 - BERTopic - Transformed documents to Embeddings
2022-02-20 14:51:23,079 - BERTopic - Reduced dimensionality with UMAP
2022-02-20 14:51:23,233 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2022-02-20 14:51:30,054 - BERTopic - Reduced number of topics from 54 to 31


In [45]:
topic_model_v2.get_topic_freq()

,Topic,Count
0,-1,359
1,0,304
2,1,178
3,2,134
4,3,132
5,4,45
6,5,42
7,6,37
8,7,36
9,8,33


In [46]:
pedidos_lgpd['requests_topics_answers'] = requests_topics
pedidos_lgpd['requests_probabilities_answers'] = requests_probabilities

In [49]:
for t in set(requests_topics):
    pedidos_lgpd.loc[pedidos_lgpd.requests_topics_answers == t,
                              'requests_topics_top_words_answers'] = \
    str([a_tuple[0] for a_tuple in topic_model_v2.get_topic(t)])[1:-1]

In [50]:
pedidos_lgpd[['requests_topics_answers',
              'requests_topics_top_words_answers']].drop_duplicates().sort_values('requests_topics_answers')

,requests_topics_answers,requests_topics_top_words_answers
63362,-1,"'de', 'que', 'da', 'do', 'dados', 'em', 'ou', 'no', 'lei', 'se'"
188117,0,"'incra', 'br', 'do', 'acesso', 'link', 'sei', 'acessando', 'mail', 'cadastro', 'termo'"
101258,1,"'de', 'dados', 'ou', 'que', 'da', 'em', 'do', 'pessoais', 'lei', 'os'"
297990,2,"'casos', 'covid', '19', 'saude', 'síndrome', 'de', 'vigilância', 'saúde', 'ministério', 'influenza'"
187872,3,"'inss', 'meu', 'central', 'telefone', 'inc', 'atendimento', '135', 'serviço', 'informações', 'criado'"
183844,4,"'imbel', 'ouvidoria', 'da', 'de', 'que', 'manifestação', 'publicidade', 'com', 'trabalho', 'no'"
66934,5,"'república', 'presidente', 'da', 'segurança', 'presidência', 'do', 'de', 'art', 'gabinete', 'que'"
65822,6,"'anpd', '2021', 'de', 'autoridade', 'proteção', 'da', '01', 'lgpd', 'que', 'nacional'"
191727,7,"'república', 'presidente', 'segurança', 'da', 'presidência', 'do', 'vice', 'tratamento', 'art', 'gabinete'"
111649,8,"'inep', 'de', 'educacionais', 'que', 'da', 'do', 'informação', 'microdados', 'dos', 'ufrgs'"


In [77]:
pedidos_lgpd.to_csv('../data/pedidos_lgpd.csv',index=False)

In [78]:
topic_model_v2.get_topic(3)

[('inss', 0.09409252617362487),
 ('meu', 0.0708985342603696),
 ('central', 0.0455746086910481),
 ('telefone', 0.042694756773734514),
 ('inc', 0.04195539290749313),
 ('atendimento', 0.040983334183268445),
 ('135', 0.04019638739734234),
 ('serviço', 0.039686261173998524),
 ('informações', 0.039593154840608234),
 ('criado', 0.031558265058684275)]